In [1]:
import numpy as np, healpy as hp, curvedsky, prjlib, plottools as pl
from matplotlib.pyplot import *
%matplotlib inline

In [2]:
pL  = prjlib.init_analysis(wtype='Lmask')
pG  = prjlib.init_analysis(wtype='G60')
pGL = prjlib.init_analysis(wtype='G60Lmask')
nside = 2048
p  = prjlib.init_analysis()
cy = prjlib.init_compy(p.ids)

### Galactic mask

In [3]:
Gmap = {}
for n in range(5):
    Gmap[str(n)] = hp.fitsfunc.read_map(p.fmask_Gfsky,field=n,verbose=False)
    print(np.mean(Gmap[str(n)]))
pl.view_maps(Gmap,min=0,max=1)

/global/homes/t/toshiyan/.conda/envs/loc/lib/python3.7/site-packages/healpy/fitsfunc.py:367: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  warnings.warn("No INDXSCHM keyword in header file : " "assume {}".format(schm))


0.20479756593704224
0.40292394161224365
0.600813110669454
0.7004634737968445
0.8000455101331075
map key list: ['0', '1', '2', '3', '4']


### ymap mask

In [4]:
Gmap = {}
for n in range(5):
    Gmap[str(n)] = hp.fitsfunc.read_map(cy.fymask_org,field=n,verbose=False)
    print(np.mean(Gmap[str(n)]))
pl.view_maps(Gmap,min=0,max=1)

0.5995168577297865
0.499804867503258
0.3989443509380098
0.2993510582951348
0.7600978414217631
map key list: ['0', '1', '2', '3', '4']


In [5]:
ymask = np.zeros((5,12*nside**2))
yptsr = hp.fitsfunc.read_map(cy.fymask_org,field=4,verbose=False)
for yn in [0,1,2]:
    ymask[yn,:] = hp.fitsfunc.read_map(cy.fymask_org,field=yn,verbose=False)
    ymask[yn,:] *= yptsr
ymask[3,:] = ymask[0,:]*prjlib.theta_mask(nside,75.)
ymask[4,:] = ymask[0,:]*prjlib.theta_mask(nside,60.)
#hp.fitsfunc.write_map(cy.fymask,ymask,overwrite=True) # save

0.9659258262890683
0.8660254037844387


In [6]:
aymask = np.zeros((5,12*nside**2))
# apodization
apmask = curvedsky.utils.apodize(nside,yptsr,0.5)
for n in range(3):
    ymask = hp.fitsfunc.read_map(cy.fymask_org,field=n,verbose=False)
    aymask[n,:] = curvedsky.utils.apodize(nside,ymask,1.0)
    aymask[n,:] *= apmask
cmask = prjlib.theta_mask(nside,75.)
acmask = curvedsky.utils.apodize(nside,cmask,1.0)
aymask[3,:] = aymask[0,:] * cmask
cmask = prjlib.theta_mask(nside,60.)
acmask = curvedsky.utils.apodize(nside,cmask,1.0)
aymask[4,:] = aymask[0,:] * cmask
hp.fitsfunc.write_map(cy.faymask,aymask,overwrite=True) # save

0.9659258262890683
0.8660254037844387


In [8]:
Gmap = {}
for n in range(5):
    Gmap[str(n)] = aymask[n,:]
    print(np.mean(aymask[n,:]))
pl.view_maps(Gmap,min=0,max=1)

0.28710260263102017
0.24515418361109395
0.19925557980500397
0.2792493100524365
0.25561975910913715
map key list: ['0', '1', '2', '3', '4']
